In [3]:
import mysql.connector
# library for data analysis
import pandas as pd

from sklearn.model_selection import train_test_split 
from sklearn.metrics.pairwise import cosine_similarity

import os 

In [5]:
# Membuat koneksi ke database
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='tourista_db'
)

In [7]:
# Mengeksekusi query untuk mengambil data dari tabel

query = "SELECT * FROM destination"
destination = pd.read_sql_query(query, conn)

query = "SELECT * FROM review_wisata"
ratings = pd.read_sql_query(query, conn)

query = "SELECT * FROM user_profile"
users = pd.read_sql_query(query, conn)

C:\Users\dewan\AppData\Local\Temp\ipykernel_10164\707316895.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  destination = pd.read_sql_query(query, conn)
C:\Users\dewan\AppData\Local\Temp\ipykernel_10164\707316895.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ratings = pd.read_sql_query(query, conn)
C:\Users\dewan\AppData\Local\Temp\ipykernel_10164\707316895.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  users = pd.read_sql_query(query, conn)


In [8]:
destination.head(2)

,id,name_wisata,description_wisata,category,destination_photo,city,price,rating,time_minutes,coordinate,destination_lat,destination_long
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,lala.png,Jakarta,20000.0,4.6,15.0,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.17539,106.8272
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,lala.png,Jakarta,5000.0,4.6,90.0,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.13764,106.8171


In [22]:
ratings.head(2)

,user_wisata,wisata_id,wisata_rating
0,1,179,3.0
1,1,344,2.0


In [26]:
ratings = ratings.rename(columns={'user_wisata': 'id'})

In [27]:
ratings.head(2)

,id,wisata_id,wisata_rating
0,1,179,3.0
1,1,344,2.0


In [34]:
users.head(2)

,id,name,age,email,phone_number,address,photo_profile,user_lat,user_lot,createdAt,updatedAt
0,1,user1,20,userTrial1@gmail.com,81326812647,"Semarang, Jawa Tengah",user1.png,-11931.0,12311.0,None,None
1,2,user2,21,userTrial2@gmail.com,81326812647,"Bekasi, Jawa Barat",user2.png,-11931.0,12311.0,None,None


In [17]:
destination[['id', 'name_wisata', 'category', 'description_wisata']]

,id,name_wisata,category,description_wisata
0,1,Monumen Nasional,Budaya,Monumen Nasional atau yang populer disingkat d...
1,2,Kota Tua,Budaya,"Kota tua di Jakarta, yang juga bernama Kota Tu..."
2,3,Dunia Fantasi,Taman Hiburan,Dunia Fantasi atau disebut juga Dufan adalah t...
3,4,Taman Mini Indonesia Indah (TMII),Taman Hiburan,Taman Mini Indonesia Indah merupakan suatu kaw...
4,5,Atlantis Water Adventure,Taman Hiburan,Atlantis Water Adventure atau dikenal dengan A...
...,...,...,...,...
432,433,Museum Mpu Tantular,Budaya,Museum Negeri Mpu Tantular adalah sebuah museu...
433,434,Taman Bungkul,Taman Hiburan,Taman Bungkul adalah taman wisata kota yang te...
434,435,Taman Air Mancur Menari Kenjeran,Taman Hiburan,Air mancur menari atau dancing fountain juga a...
435,436,Taman Flora Bratang Surabaya,Taman Hiburan,Taman Flora adalah salah satu taman kota di Su...


In [35]:
# merge rating dataset with destination dataset on Place_Id column 
full = pd.merge(ratings, destination[['id', 'name_wisata', 'category', 'description_wisata']], on = 'id',how = 'left')

# merge full dataset (destination & rating) with user dataset (age's column)
merged_df = pd.merge(full, users, on='id')

In [37]:
preapare = merged_df
preapare.head(2)

,id,wisata_id,wisata_rating,name_wisata,category,description_wisata,name,age,email,phone_number,address,photo_profile,user_lat,user_lot,createdAt,updatedAt
0,1,179,3.0,Monumen Nasional,Budaya,Monumen Nasional atau yang populer disingkat d...,user1,20,userTrial1@gmail.com,81326812647,"Semarang, Jawa Tengah",user1.png,-11931.0,12311.0,None,None
29,1,328,2.0,Monumen Nasional,Budaya,Monumen Nasional atau yang populer disingkat d...,user1,20,userTrial1@gmail.com,81326812647,"Semarang, Jawa Tengah",user1.png,-11931.0,12311.0,None,None


In [38]:
preapare = merged_df

# sort merged_df by Place_Id
merged_df = merged_df.sort_values('id')
# drop duplicates 
merged_df = merged_df.sort_values('id', ascending = True).drop_duplicates(subset = ['wisata_id', 'id'])

In [39]:
# new variable for content based filtering 
place_id = destination['Place_Id'].unique().tolist()
place_name = destination['Place_Name'].unique().tolist()
place_category = destination['Category'].tolist()

KeyError: 'Place_Id'